In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)

In [ ]:
t_cells <- readRDS('../../03_scRNA_Comparisons/Pipeline/clean_aging_t_cells.rds')

In [ ]:
t_cells

In [ ]:
table(t_cells$age)

In [ ]:
unique(t_cells$pbmc_sample_id)

In [ ]:
t_cells <- SetIdent(t_cells, value = 'pbmc_sample_id')
t_cells <- RenameIdents(t_cells,
                        'PB00892-01'='M','PB00833-01'='M','PB00913-01'='M','PB00928-01'='M','PB00809-01'='F',
                        'PB00842-01'='M','PB00914-01'='M','PB00841-01'='F','PB00127-01'='F','PB00910-01'='F',
                        'PB00182-01'='F','PB00192-01'='F','PB00811-01'='F','PB00173-01'='F','PB00884-01'='M',
                        'PB00193-01'='F','PB00026-05'='M','PB00010-02'='M','PB00003-01'='F',
                        'PB00031-05'='F','PB00507-01'='F','PB00002-01'='M','PB00384-01'='M',
                        'PB00006-01'='F','PB00023-05'='F','PB00041-01'='M',
                        'PB00022-01'='F','PB00363-01'='M','PB00386-01'='M','PB00366-01'='F',
                        'PB00030-03'='F','PB00004-01'='M','PB00552-01'='F','PB00356-01'='M',
                        'PB00530-01'='F','PB00510-01'='M','PB00352-04'='F','PB00015-01'='M',
                        'PB00388-01'='F','PB00539-01'='M','PB00569-01'='F','PB00342-01'='M','PB00566-01'='M',
                        'PB00537-01'='F','PB00561-01'='F','PB00562-01'='M','PB00568-01'='F',
                        'PB00582-01'='M')
t_cells$sex <- Idents(t_cells)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
DimPlot(t_cells, label = TRUE, repel = TRUE, group.by = 'SCT_snn_res.0.5')
DimPlot(t_cells, label = TRUE, repel = TRUE, group.by = 'sex')

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
DimPlot(t_cells, split.by = 'batch_id', ncol = 4, group.by = 'SCT_snn_res.0.5')

In [ ]:
t_cells <- SetIdent(t_cells, value = 'SCT_snn_res.0.5')
t_cells <- FindSubCluster(object = t_cells, cluster = 11, graph.name = 'SCT_snn', resolution = 0.2)
Idents(t_cells) <- "sub.cluster"

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(t_cells, label = TRUE, repel = TRUE)
table(t_cells$sub.cluster)

In [ ]:
DefaultAssay(t_cells) <- 'RNA'
t_cells <- NormalizeData(t_cells)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)
VlnPlot(t_cells, features = c('KLRC2','ZNF683','CD3E','CD3G','TRGC2','TRDC',
                              'IKZF2','LEF1'), ncol = 2, pt.size = 0)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 12)
FeaturePlot(t_cells, features = c('KLRC2','ZNF683','IKZF2',
                                  'TRGC1','TRGC2','TRDC',
                                  'IL21R','CD8A','DUSP2'), ncol = 3, order = TRUE)

In [ ]:
mnp2_markers <- FindMarkers(t_cells, `ident.1` = '11_1', max.cells.per.ident = 500)

In [ ]:
head(subset(mnp2_markers, avg_log2FC > 0), n=20)

In [ ]:
ped_gdT_markers <- FindMarkers(t_cells, `ident.1` = '11_0', max.cells.per.ident = 500)

In [ ]:
head(subset(ped_gdT_markers, avg_log2FC > 0), n=20)

In [ ]:
cluster12_2 <- FindMarkers(t_cells, `ident.1` = '11_2', max.cells.per.ident = 500)

In [ ]:
head(subset(cluster12_2, avg_log2FC > 0), n=20)

In [ ]:
t_cells <- SetIdent(t_cells, value = 'age')
subset_50k <- subset(t_cells, downsample = 75000)

In [ ]:
p <- plot_density(subset_50k, c('IL21R','LEF1','KLRC2'), joint = TRUE)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
p

In [ ]:
p <- plot_density(subset_50k, c('LEF1','DUSP2'), joint = TRUE)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
p + plot_layout(ncol = 2)

In [ ]:
p <- plot_density(subset_50k, c('LEF1','KLRC3','CD8A'), joint = TRUE)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
p + plot_layout(ncol = 2)

In [ ]:
# Open a pdf file
pdf("plots/mnp2_pipeline_markers_nebulosa_umap.pdf", width = 12, height = 12) 
# 2. Create a plot
p
# Close the pdf file
dev.off() 

In [ ]:
# 'KLRC2','ZNF683','IKZF2',
# 'TRGC1','TRGC2','TRDC',
# 'IL21R','CD8A','DUSP2

In [ ]:
p[[4]]

In [ ]:
t_cells <- SetIdent(t_cells, value = 'sub.cluster')
mnp2_cluster <- subset(t_cells, idents = '11_1')

In [ ]:
mnp2_counts <- dplyr::count(mnp2_cluster@meta.data, pbmc_sample_id, age, sex)
head(mnp2_counts)

In [ ]:
total_t_counts <- dplyr::count(t_cells@meta.data, pbmc_sample_id)
head(total_t_counts)

In [ ]:
joined_counts <- left_join(mnp2_counts, total_t_counts, by = 'pbmc_sample_id')
head(joined_counts)

In [ ]:
joined_counts$perc <- joined_counts$n.x / joined_counts$n.y
head(joined_counts)

In [ ]:
write.csv(joined_counts, file = 'mnp2_scrna_seq_perc.csv')

In [ ]:
joined_counts <- read.csv(file = 'mnp2_scrna_seq_perc_age.csv')

In [ ]:
head(joined_counts)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
ggplot(joined_counts, aes(x=age, y=perc, color=sex)) + 
    geom_point(size = 3) + 
    geom_smooth(formula=y ~ x^2,alpha=.15, aes(fill=sex), se = FALSE) + 
    scale_color_manual(values=c('#08519c','#9ecae1')) + 
    xlab('Age') + ylab('Fraction MNP-2 of Total T') +
    theme_bw() + 
    theme(axis.text.x = element_text(size = 15),
          axis.text.y = element_text(size = 15),
          axis.title.y = element_text(size = 20),
          axis.title.x = element_text(size = 20),
          # legend.key.size = unit(2, 'cm'), #change legend key size
          # legend.key.height = unit(1, 'cm'), #change legend key height
          # legend.key.width = unit(1, 'cm'), #change legend key width
          legend.title = element_text(size=20), #change legend title font size
          legend.text = element_text(size=15))

In [ ]:
# Open a pdf file
pdf("plots/mnp2_pipeline_age_sex_dotplot.pdf", width = 8, height = 8) 
# 2. Create a plot
ggplot(joined_counts, aes(x=age, y=perc, color=sex)) + 
    geom_point(size = 3) + 
    geom_smooth(formula=y ~ x^2,alpha=.15, aes(fill=sex), se = FALSE) + 
    scale_color_manual(values=c('#08519c','#9ecae1')) + 
    xlab('Age') + ylab('Fraction MNP-2 of Total T') +
    theme_bw() + 
    theme(axis.text.x = element_text(size = 15),
          axis.text.y = element_text(size = 15),
          axis.title.y = element_text(size = 20),
          axis.title.x = element_text(size = 20),
          # legend.key.size = unit(2, 'cm'), #change legend key size
          # legend.key.height = unit(1, 'cm'), #change legend key height
          # legend.key.width = unit(1, 'cm'), #change legend key width
          legend.title = element_text(size=20), #change legend title font size
          legend.text = element_text(size=15))
# Close the pdf file
dev.off() 